In [1]:
import os
import cv2
from PIL import Image
from ultralytics import YOLO

# Configurazione
SIZE = (640, 360)
MODEL_PATH = "/home/lorenzo/Scaricati/license_plate_detector.pt"
INPUT_DIR = "/home/lorenzo/Scaricati/OneDrive_2025-02-27/GH010217 (new)"
OUTPUT_DIR = os.path.join(INPUT_DIR, "converted_jpg")

# Carica il modello YOLO per il riconoscimento delle targhe
model = YOLO(MODEL_PATH)

os.makedirs(OUTPUT_DIR, exist_ok=True)

def process_images(directory):
    for filename in os.listdir(directory):
        if filename.lower().endswith(".jpg"):
            img_path = os.path.join(directory, filename)
            output_path = os.path.join(OUTPUT_DIR, filename)

            # Apri l'immagine con OpenCV
            image = cv2.imread(img_path)
            if image is None:
                print(f"Errore nel caricamento: {img_path}")
                continue

            # Rilevamento delle targhe con YOLO
            results = model(image)
            for result in results:
                for box in result.boxes.xyxy:
                    x1, y1, x2, y2 = map(int, box)
                    plate_roi = image[y1:y2, x1:x2]
                    blurred_roi = cv2.GaussianBlur(plate_roi, (101, 101), 0)
                    image[y1:y2, x1:x2] = blurred_roi

            # Converti l'immagine in formato PIL per ridimensionarla
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            pil_img = Image.fromarray(image)
            pil_img = pil_img.resize(SIZE)
            
            # Salva l'immagine elaborata
            pil_img.save(output_path, "JPEG", quality=95)
            print(f"Processato: {img_path} -> {output_path}")

if __name__ == "__main__":
    process_images(INPUT_DIR)
    print("Elaborazione completata!")


0: 384x640 2 license_plates, 156.7ms
Speed: 4.1ms preprocess, 156.7ms inference, 26.9ms postprocess per image at shape (1, 3, 384, 640)
Processato: /home/lorenzo/Scaricati/OneDrive_2025-02-27/GH010217 (new)/vlcsnap-2025-02-26-20h36m21s506.jpg -> /home/lorenzo/Scaricati/OneDrive_2025-02-27/GH010217 (new)/converted_jpg/vlcsnap-2025-02-26-20h36m21s506.jpg

0: 384x640 (no detections), 56.5ms
Speed: 2.4ms preprocess, 56.5ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)
Processato: /home/lorenzo/Scaricati/OneDrive_2025-02-27/GH010217 (new)/vlcsnap-2025-02-26-20h40m33s122.jpg -> /home/lorenzo/Scaricati/OneDrive_2025-02-27/GH010217 (new)/converted_jpg/vlcsnap-2025-02-26-20h40m33s122.jpg

0: 384x640 (no detections), 86.6ms
Speed: 9.7ms preprocess, 86.6ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)
Processato: /home/lorenzo/Scaricati/OneDrive_2025-02-27/GH010217 (new)/vlcsnap-2025-02-26-20h37m59s545.jpg -> /home/lorenzo/Scaricati/OneDrive_2025-02-27/GH010

In [1]:
import os
import shutil
import random

def split_dataset(base_path, valid_ratio=0.05):
    # Definisce i path per le cartelle train e valid
    train_images = os.path.join(base_path, "train", "images")
    train_labels = os.path.join(base_path, "train", "labels")
    valid_images = os.path.join(base_path, "valid", "images")
    valid_labels = os.path.join(base_path, "valid", "labels")

    # Crea le directory di valid se non esistono
    os.makedirs(valid_images, exist_ok=True)
    os.makedirs(valid_labels, exist_ok=True)

    # Elenca tutti i file immagini in train/images
    image_files = [f for f in os.listdir(train_images) if os.path.isfile(os.path.join(train_images, f))]
    total_images = len(image_files)
    
    # Calcola quanti file spostare (almeno 1)
    num_to_move = max(1, int(total_images * valid_ratio))
    print(f"Totale immagini in train: {total_images}. Verranno spostate {num_to_move} immagini (e relativi label) in valid.")

    # Seleziona casualmente i file da spostare
    files_to_move = random.sample(image_files, num_to_move)

    for filename in files_to_move:
        # Sposta l'immagine dalla cartella train/images a valid/images
        src_img = os.path.join(train_images, filename)
        dst_img = os.path.join(valid_images, filename)
        shutil.move(src_img, dst_img)
        print(f"Spostata immagine: {src_img} -> {dst_img}")

        # Sposta il file label corrispondente se esiste
        src_label = os.path.join(train_labels, filename[:-3] + "txt")
        dst_label = os.path.join(valid_labels, filename[:-3] + "txt")
        if os.path.exists(src_label):
            shutil.move(src_label, dst_label)
            print(f"Spostato label: {src_label} -> {dst_label}")
        else:
            print(f"Nessun label trovato per l'immagine: {filename}")

# Imposta qui il percorso della cartella che contiene "train" e "valid"
path = "/home/lorenzo/Scrivania/dataset/merged/data"  # Modifica questo percorso

# Esegui lo split del dataset
split_dataset(path)

Totale immagini in train: 2009. Verranno spostate 100 immagini (e relativi label) in valid.
Spostata immagine: /home/lorenzo/Scrivania/dataset/merged/data/train/images/vlcsnap-2025-02-26-20h34m52s238.jpg -> /home/lorenzo/Scrivania/dataset/merged/data/valid/images/vlcsnap-2025-02-26-20h34m52s238.jpg
Spostato label: /home/lorenzo/Scrivania/dataset/merged/data/train/labels/vlcsnap-2025-02-26-20h34m52s238.txt -> /home/lorenzo/Scrivania/dataset/merged/data/valid/labels/vlcsnap-2025-02-26-20h34m52s238.txt
Spostata immagine: /home/lorenzo/Scrivania/dataset/merged/data/train/images/vlcsnap-2025-02-19-14h43m57s951.jpg -> /home/lorenzo/Scrivania/dataset/merged/data/valid/images/vlcsnap-2025-02-19-14h43m57s951.jpg
Spostato label: /home/lorenzo/Scrivania/dataset/merged/data/train/labels/vlcsnap-2025-02-19-14h43m57s951.txt -> /home/lorenzo/Scrivania/dataset/merged/data/valid/labels/vlcsnap-2025-02-19-14h43m57s951.txt
Spostata immagine: /home/lorenzo/Scrivania/dataset/merged/data/train/images/vlcsna

In [42]:
# Importare le librerie necessarie
import pandas as pd
import folium
from folium.plugins import MarkerCluster
from datetime import datetime
import branca.colormap as cm
import numpy as np

# Leggere il file TSV
data = pd.read_csv('detections.tsv', sep='\t')

# Convertire le colonne di timestamp in datetime
data['timestamp'] = pd.to_datetime(data['timestamp'])
data['timestamp_gps'] = pd.to_datetime(data['timestamp_gps'])

# Filtrare solo le righe con coordinate GPS valide
data_gps = data.dropna(subset=['lat', 'lon'])

# Calcolare il centro della mappa (media delle coordinate)
center_lat = data_gps['lat'].mean()
center_lon = data_gps['lon'].mean()

# Creare una mappa bella e moderna
m = folium.Map(
    location=[center_lat, center_lon],
    zoom_start=15,
    tiles='CartoDB positron',  # Stile moderno e chiaro
    max_zoom=22  # Limitare il livello massimo di zoom per evitare problemi di rendering
)

# Creare un cluster di marker che permette lo zoom automatico quando cliccato
marker_cluster = MarkerCluster(
    options={
        'spiderfyOnMaxZoom': True,     # Espande i marker quando si raggiunge lo zoom massimo
        'disableClusteringAtZoom': 18,  # Disattiva il clustering quando si zooma molto vicino
        'maxClusterRadius': 80,         # Raggio più piccolo per distinguere meglio i gruppi
        'zoomToBoundsOnClick': True     # Zooma al click sul cluster
    }
).add_to(m)

# Creare una scala di colori basata sul valore di confidenza
colormap = cm.LinearColormap(
    colors=['orange', 'red', 'darkred'],
    vmin=data_gps['confidence'].min(),
    vmax=data_gps['confidence'].max()
)

# Calcolare raggio minimo e massimo per la scala
min_area = data_gps['area'].min()
max_area = data_gps['area'].max()
min_radius = 5
max_radius = 15

# Funzione per scalare il raggio in base all'area
def scale_radius(area):
    # Normalizzazione log per avere una scala più usabile
    scaled = np.interp(
        np.log(area), 
        [np.log(min_area), np.log(max_area)], 
        [min_radius, max_radius]
    )
    return scaled

# Aggiungere punti sulla mappa per ogni detection con coordinate GPS
for idx, row in data_gps.iterrows():
    # Calcolare il colore in base alla confidenza
    color = colormap(row['confidence'])
    
    # Calcolare il raggio in base all'area
    radius = scale_radius(row['area'])
    
    # Formattare il popup con informazioni utili
    popup_text = f"""
    <div style="font-family: Arial, sans-serif; width: 250px;">
        <h4 style="margin-bottom: 5px;">Detection Details</h4>
        <hr style="margin: 5px 0;">
        <b>Type:</b> {row['type']}<br>
        <b>Area:</b> {row['area']} px²<br>
        <b>Confidence:</b> {row['confidence']:.2f}<br>
        <b>Time:</b> {row['timestamp'].strftime('%Y-%m-%d %H:%M:%S')}<br>
        <b>Location:</b> {row['lat']:.6f}, {row['lon']:.6f}
    </div>
    """
    
    # Creare un cerchio colorato per ogni detection
    folium.CircleMarker(
        location=[row['lat'], row['lon']],
        radius=radius,  # Raggio proporzionale all'area
        popup=folium.Popup(popup_text, max_width=300),
        color=color,
        fill=True,
        fill_color=color,
        fill_opacity=0.7,
        weight=2,
        tooltip=f"Detection {idx+1}: Area {row['area']} px²"
    ).add_to(marker_cluster)

# Aggiungere la legenda per la scala di colori
colormap.caption = 'Confidence Level'
colormap.add_to(m)

# Aggiungere controlli per la visualizzazione di diverse mappe base
# Utilizzare mappe che supportano livelli di zoom elevati

# Mappa base di default
folium.TileLayer(
    'CartoDB positron', 
    name='Light Mode (Default)',
    max_zoom=19,
).add_to(m)

# Opzioni alternative di mappa
folium.TileLayer(
    'CartoDB dark_matter', 
    name='Dark Mode',
    max_zoom=19,
).add_to(m)

folium.TileLayer(
    'OpenStreetMap', 
    name='OpenStreetMap',
    attr='&copy; <a href="https://www.openstreetmap.org/copyright">OpenStreetMap</a> contributors',
    max_zoom=19,
).add_to(m)

# Aggiungiamo una legenda per i raggi
radius_legend_html = f'''
<div style="position: fixed; 
            top: 50px; right: 50px; 
            border:2px solid grey; z-index:9999; font-size:14px;
            background-color:white;
            padding: 10px;
            border-radius: 5px;
            opacity: 0.8;
            ">
    <h4 style="margin-top: 0;">Area Legend</h4>
    <div style="display: flex; align-items: center; margin-bottom: 5px;">
        <div style="background-color: red; border-radius: 50%; width: {min_radius*2}px; height: {min_radius*2}px; margin-right: 10px;"></div>
        <span>Small area (~{min_area:.0f} px²)</span>
    </div>
    <div style="display: flex; align-items: center; margin-bottom: 5px;">
        <div style="background-color: red; border-radius: 50%; width: {max_radius*2}px; height: {max_radius*2}px; margin-right: 10px;"></div>
        <span>Large area (~{max_area:.0f} px²)</span>
    </div>
    <div style="font-style: italic; font-size: 12px; margin-top: 10px;">
        Circle size is proportional to detection area
    </div>
</div>
'''
m.get_root().html.add_child(folium.Element(radius_legend_html))

# Aggiungere una legenda per i tipi di detection
types_legend_html = '''
<div style="position: fixed; 
            bottom: 50px; right: 50px; 
            border:2px solid grey; z-index:9999; font-size:14px;
            background-color:white;
            padding: 10px;
            border-radius: 5px;
            opacity: 0.8;
            ">
    <h4 style="margin-top: 0;">Detection Types</h4>
    <p><i class="fa fa-circle" style="color:red"></i> Face Detection</p>
    <div style="font-style: italic; font-size: 12px; margin-top: 10px;">
        Click on clusters to zoom in and see individual detections.
    </div>
</div>
'''
m.get_root().html.add_child(folium.Element(types_legend_html))

# Aggiungere controlli per i layer
folium.LayerControl().add_to(m)

# Aggiungere scala
folium.plugins.MeasureControl(position='bottomleft', primary_length_unit='meters').add_to(m)

# Aggiungere funzionalità di fullscreen
folium.plugins.Fullscreen(position='topleft').add_to(m)

# Aggiungere plugin per prendere le coordinate con il mouse
folium.plugins.MousePosition(
    position='bottomright',
    separator=' | ',
    prefix='Coordinates:',
    num_digits=6,
    empty_string='',
).add_to(m)

# Aggiungere una mini mappa per il contesto
minimap = folium.plugins.MiniMap(toggle_display=True)
m.add_child(minimap)

# Salvare la mappa in un file HTML
m.save('detections_map.html')

# Visualizzare la mappa nel notebook
m